In [1]:
import json
import os
import sys

In [2]:
# os.chdir('../')

In [3]:
from icecream import ic
import numpy as np
import torch
import torch.multiprocessing as mp
import torch.nn as nn
from torchsummary import summary
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.models import densenet
from tqdm import tqdm

In [4]:
sys.path.append('../dataset/')
import coco_data_prep

loading annotations into memory...
Done (t=12.20s)
creating index...
index created!
loading annotations into memory...
Done (t=6.64s)
creating index...
index created!
loading annotations into memory...
Done (t=0.12s)
creating index...
index created!


In [5]:
%load_ext autoreload

### Global Variables

In [ ]:
# train_np_data_dir = '../data/numpy_imgs/train_subset/'
# train_jpg_data_dir = '../data/raw/train/train2014/'
# train_annot_filepath = '../data/raw/train/annotations/instances_train2014.json'

# with open('../dataset/imgs_by_supercategory.json', 'r') as f:
#     desired_categories = json.load(f)

In [6]:
val_np_data_dir = '../data/numpy_imgs/val_subset/'
val_jpg_data_dir = '../data/raw/val/val2014/'
val_annot_filepath = '../data/raw/train/annotations/instances_val2014.json'

In [7]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device('cpu')

### Load Data

In [8]:
%autoreload

In [9]:
all_val_ds = coco_data_prep.COCODataset('val',
                                        val_np_data_dir, 
                                        val_annot_filepath,
                                        coco_data_prep.COCOAnnotationTransform(),
                                        device='cpu')

loading annotations into memory...
Done (t=6.10s)
creating index...
index created!


100%|███████████████████████████████████████████| 40504/40504 [00:32<00:00, 1261.99it/s]


In [10]:
len(all_val_ds)

21002

In [11]:
all_val_dl = coco_data_prep.get_dataloader(all_val_ds, 
                                         batch_size=1, 
                                         device='cpu',
                                         loader_params={'num_workers': 0,
                                                        'shuffle': False,
                                                        'collate_fn': lambda x : x})

In [12]:
len(all_val_dl)

21002

### Load Model

#### Densenet121

In [34]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'densenet121', pretrained=True)

Using cache found in /home/ec2-user/.cache/torch/hub/pytorch_vision_v0.10.0


In [14]:
def slice_model(original_model, from_layer=None, to_layer=None):
    return nn.Sequential(*list(original_model.children())[from_layer:to_layer])

In [16]:
model_conv_features = slice_model(model, to_layer=-1).to(device)
# model_2 = torch.nn.DataParallel(model_conv_features, device_ids=[0,1]).cuda()

In [18]:
img_ids_in_order = [all_val_ds.ids[i] for i in range(len(all_val_ds))]

In [19]:
densenet_index2cocoID = dict(zip(range(len(all_val_ds)), img_ids_in_order))
with open ('../data/torch_embeddings/densenet_pretrained/densenet_index2cocoIDs.json', 'w') as f:
    json.dump(densenet_index2cocoID, f)

In [24]:
features_list = []

# Use GPUs to speed up the inference, this should take around 10 minutes
model_conv_features.to('cpu')

for i, batch in tqdm(enumerate(all_val_dl)):    
    image_batch, _ = [x[0] for x in batch], [x[1] for x in batch]
    image_batch = torch.stack(image_batch).to('cpu')

    with torch.no_grad():
        features_list.append(model_conv_features(image_batch).flatten(start_dim=1))
        

21002it [42:03,  8.32it/s]


In [25]:
embeddings = torch.cat([b for b in features_list], dim=0)

In [26]:
torch.save(embeddings, f'../data/torch_embeddings/densenet_pretrained/densenet_embed_validation.pt')